In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# PhonePe Pulse API Ingestion – Insurance (Aggregated)

This notebook performs API-based ingestion of aggregated insurance data from the official PhonePe Pulse repository.

Objective:
- Fetch quarterly insurance JSON files
- Parse nested policy metrics
- Convert them into structured tabular format
- Export clean CSV for warehouse loading

Insurance data starts from 2020 Q2 as per the public repository.


In [2]:
import os
import sys

print("Python version:", sys.version)
print("Working directory:", os.getcwd())


Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Working directory: /kaggle/working


## 1. Import Required Libraries

In this step, we import the necessary Python libraries for:

- Making API requests
- Handling JSON responses
- Data transformation
- DataFrame creation
- Exporting structured CSV

These libraries support the ingestion process.


In [3]:
import requests
import json
import pandas as pd
import os
from time import sleep


## 2. Define Insurance API Source

Here we define the base GitHub raw URL for the aggregated insurance dataset.

Source:
Official PhonePe Pulse public repository.

Dataset Path:
aggregated/insurance/country/india/

This dataset provides national-level quarterly insurance metrics.


In [4]:
INSURANCE_BASE_URL = "https://raw.githubusercontent.com/PhonePe/pulse/master/data/aggregated/insurance/country/india"


In [5]:
START_YEAR = 2018
END_YEAR = 2024
QUARTERS = [1, 2, 3, 4]


In [6]:
INSURANCE_RAW_DIR = "/kaggle/working/insurance_raw"
os.makedirs(INSURANCE_RAW_DIR, exist_ok=True)

print("Insurance raw directory:", INSURANCE_RAW_DIR)


Insurance raw directory: /kaggle/working/insurance_raw


In [7]:
import requests

url_missing = "https://raw.githubusercontent.com/PhonePe/pulse/master/data/aggregated/insurance/country/india/2020/1.json"
r = requests.get(url_missing)
print("Status code (2020 Q1):", r.status_code)


Status code (2020 Q1): 404


## 3. Define Time Coverage (Insurance Dataset)

Insurance data is available starting from 2020 Q2.

We define:
- Year range
- Quarter iteration

The loop dynamically constructs URLs for each available quarter.


In [8]:
url_valid = "https://raw.githubusercontent.com/PhonePe/pulse/master/data/aggregated/insurance/country/india/2020/2.json"
r = requests.get(url_valid)

print("Status code (2020 Q2):", r.status_code)
print("Data keys:", r.json()["data"].keys())


Status code (2020 Q2): 200
Data keys: dict_keys(['from', 'to', 'transactionData'])


In [9]:
data = r.json()

data["data"]["transactionData"]


[{'name': 'Insurance',
  'paymentInstruments': [{'type': 'TOTAL',
    'count': 185348,
    'amount': 33732166.0}]}]

## 4. Fetch Insurance JSON Data

For each year and quarter:
- Construct API URL
- Send request
- Validate response
- Parse JSON data

Only successful responses are processed.

This ensures correct ingestion without breaking on missing quarters.


In [10]:
import os
import json
import requests

INSURANCE_BASE_URL = (
    "https://raw.githubusercontent.com/PhonePe/pulse/master/"
    "data/aggregated/insurance/country/india"
)

failed_requests = []
saved_files = 0

for year in range(START_YEAR, END_YEAR + 1):
    for quarter in QUARTERS:
        url = f"{INSURANCE_BASE_URL}/{year}/{quarter}.json"
        filename = f"insurance_india_{year}_Q{quarter}.json"
        filepath = os.path.join(INSURANCE_RAW_DIR, filename)

        r = requests.get(url)

        if r.status_code == 200:
            with open(filepath, "w") as f:
                json.dump(r.json(), f)
            saved_files += 1
        else:
            failed_requests.append({
                "year": year,
                "quarter": quarter,
                "status": r.status_code
            })

print("Insurance files saved:", saved_files)
print("Missing quarters:", failed_requests)


Insurance files saved: 19
Missing quarters: [{'year': 2018, 'quarter': 1, 'status': 404}, {'year': 2018, 'quarter': 2, 'status': 404}, {'year': 2018, 'quarter': 3, 'status': 404}, {'year': 2018, 'quarter': 4, 'status': 404}, {'year': 2019, 'quarter': 1, 'status': 404}, {'year': 2019, 'quarter': 2, 'status': 404}, {'year': 2019, 'quarter': 3, 'status': 404}, {'year': 2019, 'quarter': 4, 'status': 404}, {'year': 2020, 'quarter': 1, 'status': 404}]


In [11]:
files = os.listdir(INSURANCE_RAW_DIR)
print("Total insurance JSON files:", len(files))
print("Sample files:", sorted(files)[:5])


Total insurance JSON files: 19
Sample files: ['insurance_india_2020_Q2.json', 'insurance_india_2020_Q3.json', 'insurance_india_2020_Q4.json', 'insurance_india_2021_Q1.json', 'insurance_india_2021_Q2.json']


## 5. Inspect Insurance JSON Structure

Before flattening, we inspect the structure of a sample JSON file.

The insurance dataset contains:
- Policy count
- Insurance amount
- Insurance type (if available)

This helps identify the correct parsing keys.


In [12]:
sample_file = os.listdir(INSURANCE_RAW_DIR)[0]

with open(os.path.join(INSURANCE_RAW_DIR, sample_file), "r") as f:
    sample_json = json.load(f)

sample_json["data"]["transactionData"][0]


{'name': 'Insurance',
 'paymentInstruments': [{'type': 'TOTAL',
   'count': 630758,
   'amount': 857034846.0}]}

## 6. Convert Parsed Records into DataFrame

After extracting structured records, we convert them into a Pandas DataFrame.

This format enables:
- Validation
- Standardization
- Export to warehouse-ready CSV


In [13]:
records = []

for file in os.listdir(INSURANCE_RAW_DIR):
    path = os.path.join(INSURANCE_RAW_DIR, file)

    year = int(file.split("_")[2])
    quarter = int(file.split("_Q")[1].split(".")[0])

    with open(path, "r") as f:
        data = json.load(f)

    for item in data["data"]["transactionData"]:
        pi = item["paymentInstruments"][0]

        records.append({
            "year": year,
            "quarter": quarter,
            "insurance_type": item["name"],
            "policy_count": pi["count"],              
            "insurance_amount_rupees": pi["amount"]   
        })

insurance_df = pd.DataFrame(records)


## 7. Perform Data Validation Checks

We validate:

- Row count
- Column structure
- Data types
- Missing values (if any)

This ensures ingestion integrity before export.


In [14]:
insurance_df.head()

,year,quarter,insurance_type,policy_count,insurance_amount_rupees
0,2022,2,Insurance,630758,8.570348e+08
1,2021,2,Insurance,363989,2.950667e+08
2,2024,2,Insurance,1105041,1.759628e+09
3,2024,3,Insurance,1213629,1.941266e+09
4,2022,1,Insurance,617606,8.874472e+08


In [15]:
insurance_df.shape

(19, 5)

In [16]:
insurance_df.isna().sum()

year                       0
quarter                    0
insurance_type             0
policy_count               0
insurance_amount_rupees    0
dtype: int64

## 8. Create Derived Insurance Metrics

To improve analytical usability:

- Convert insurance amount to Crores
- Calculate average policy value (amount / policy count)

These transformations standardize the dataset for SQL analysis.


In [17]:
insurance_df["insurance_amount_crore"] = (
    insurance_df["insurance_amount_rupees"] / 1e7
)



In [18]:
insurance_df["avg_policy_value"] = (
    insurance_df["insurance_amount_rupees"] / insurance_df["policy_count"]
)

In [19]:
insurance_df.shape

(19, 7)

In [20]:
insurance_df.head()

,year,quarter,insurance_type,policy_count,insurance_amount_rupees,insurance_amount_crore,avg_policy_value
0,2022,2,Insurance,630758,8.570348e+08,85.703485,1358.737972
1,2021,2,Insurance,363989,2.950667e+08,29.506668,810.647239
2,2024,2,Insurance,1105041,1.759628e+09,175.962789,1592.364350
3,2024,3,Insurance,1213629,1.941266e+09,194.126564,1599.554428
4,2022,1,Insurance,617606,8.874472e+08,88.744724,1436.914863


In [21]:
insurance_df = (
    insurance_df
    .sort_values(by=["year", "quarter", "insurance_type"])
    .reset_index(drop=True)
)


In [22]:
insurance_df.head(10)

,year,quarter,insurance_type,policy_count,insurance_amount_rupees,insurance_amount_crore,avg_policy_value
0,2020,2,Insurance,185348,3.373217e+07,3.373217,181.993688
1,2020,3,Insurance,354284,8.949508e+07,8.949508,252.608292
2,2020,4,Insurance,248626,1.709799e+08,17.097993,687.699328
3,2021,1,Insurance,318119,2.063070e+08,20.630702,648.521541
4,2021,2,Insurance,363989,2.950667e+08,29.506668,810.647239
5,2021,3,Insurance,374867,3.424034e+08,34.240335,913.399558
6,2021,4,Insurance,526327,6.554319e+08,65.543188,1245.294042
7,2022,1,Insurance,617606,8.874472e+08,88.744724,1436.914863
8,2022,2,Insurance,630758,8.570348e+08,85.703485,1358.737972
9,2022,3,Insurance,806491,1.054718e+09,105.471787,1307.786285


In [23]:
insurance_df.tail(10)

,year,quarter,insurance_type,policy_count,insurance_amount_rupees,insurance_amount_crore,avg_policy_value
9,2022,3,Insurance,806491,1.054718e+09,105.471787,1307.786285
10,2022,4,Insurance,1035364,1.439763e+09,143.976274,1390.586054
11,2023,1,Insurance,923173,1.408801e+09,140.880122,1526.042490
12,2023,2,Insurance,893850,1.347400e+09,134.739987,1507.411616
13,2023,3,Insurance,1010211,1.445235e+09,144.523468,1430.626552
14,2023,4,Insurance,1159063,1.855300e+09,185.529977,1600.689321
15,2024,1,Insurance,1273789,1.931956e+09,193.195565,1516.699904
16,2024,2,Insurance,1105041,1.759628e+09,175.962789,1592.364350
17,2024,3,Insurance,1213629,1.941266e+09,194.126564,1599.554428
18,2024,4,Insurance,1475429,2.293956e+09,229.395629,1554.772399


## 9. Export Structured Insurance Dataset

The cleaned insurance dataset is exported as CSV.

This file will serve as input for:
- SQL warehouse loading
- Insurance growth analysis
- Future financial inclusion dashboards


In [24]:
insurance_df.to_csv(
    "/kaggle/working/phonepe_insurance_aggregated_india.csv",
    index=False
)


## Conclusion

This notebook successfully:

- Retrieved quarterly insurance data
- Flattened nested JSON structure
- Validated dataset integrity
- Generated warehouse-ready CSV

This completes the insurance ingestion phase of the project.
